In [1]:
import os
import json
import yaml
import numpy as np

# In this part we extract params after hypopt in the format used for scripts

In [2]:
path = '../../workdir/run_train_models/distilbert_hypopt/metric_sn_fixed_range_last_last_fix/'
# for metric loss on electra
path = '../../workdir/run_train_models/metric_electra_hypopt_params_fix_new_lamb_171'
path = '../../workdir/run_train_models/metric_electra_hypopt_params_fix_new_lamb'
# all for distilbert
path = '../../workdir/run_train_models/distilbert_hypopt/raw_sn'

path = '../../workdir/run_train_models/distilbert_hypopt/metric_fixed_range_last_last_fix_171/'


In [18]:
path = '../../workdir/run_train_models/mixup_electra_hypopt_after_repro'
#path = '../../workdir/run_train_models/mixup_deberta_hypopt'
#path = '../../workdir/run_train_models/mixup_distilbert_hypopt'
path = '../../workdir/run_train_models/metric_electra_hypopt_params_50'
path = '../../workdir/run_train_models/raw_electra_hypopt_params_sngp'


datasets = ['mrpc', 'cola', 'sst2', 'conll']
datasets_names = {'mrpc': 'MRPC',
                  'cola': 'CoLA',
                  'sst2': 'SST-2',
                  'conll': 'CoNLL-2003'}
regs_names = {'raw': '--', 'raw_sn': '--', 'cer': 'CER', 'cer_sn': 'CER', 'metric': 'Metric', 'metric_sn': 'Metric'}
#datasets = ['conll']
regs = ['metric', 'metric_sn']
regs = ['']
regs_names = {'': '--'}
params = {'lam1': 'mixup.lam1',
          'lam2': 'mixup.lam2',
          'omega': 'mixup.omega',
          'mixup': 'mixup.mixup',
          'self_ensembling': 'mixup.self_ensembling',          
          'learning_rate': 'training.learning_rate',
          'num_train_epochs': 'training.num_train_epochs',
          'per_device_train_batch_size': 'training.per_device_train_batch_size',
          'weight_decay': '+training.weight_decay',
}

In [19]:
datasets = ['mrpc', 'cola', 'sst2', 'conll']
datasets_names = {'mrpc': 'MRPC',
                  'cola': 'CoLA',
                  'sst2': 'SST-2',
                  'conll': 'CoNLL-2003'}
regs_names = {'raw': '--', 'raw_sn': '--', 'cer': 'CER', 'cer_sn': 'CER', 'metric': 'Metric', 'metric_sn': 'Metric'}
#datasets = ['conll']
regs = ['metric', 'metric_sn']
regs = ['']
regs_names = {'': '--'}
params = {'lamb': 'ue.lamb',
          'lamb_intra': '+ue.lamb_intra',
          'learning_rate': 'training.learning_rate',
          'margin': '+ue.margin',
          'num_train_epochs': 'training.num_train_epochs',
          'per_device_train_batch_size': 'training.per_device_train_batch_size',
          'weight_decay': '+training.weight_decay',
}

In [21]:
for dataset in datasets:
    for reg in regs:
        buf_path = os.path.join(path, dataset, reg)
        params_string = ''
        for pat, subdirs, files in os.walk(buf_path):
            for name in files:
                params_string = ''
                if name == 'optimal_hyperparameters.yaml':
                    print(pat)
                    keywords = pat.split('/')[-4], pat.split('/')[-2]
                    new_path = '../../workdir/opt_params_electra_20_sngp/'
                    new_path += keywords[0] + '_' + keywords[1] + '_' + name
                    opt_path = os.path.join(pat, name)
                    with open(opt_path, 'r') as f:
                        data = yaml.safe_load(f)
                        with open(new_path, 'w') as wr_f:
                            yaml.dump(data, wr_f, default_flow_style=False)
                        for key, value in data.items():
                            param_key = params.get(key, None)
                            if param_key is not None:
                                params_string += f' {param_key}\={value}'
                        #/params_string += f' ue.reg_type\=metric'
                        print(dataset, reg, data['objective'], params_string)

../../workdir/run_train_models/raw_electra_hypopt_params_sngp/mrpc/raw/models/mrpc/23419
mrpc  0.8680479825517994  ue.lamb\=0.01 +ue.lamb_intra\=0.2 training.learning_rate\=0.0001 +ue.margin\=0.025 training.num_train_epochs\=10 training.per_device_train_batch_size\=32 +training.weight_decay\=0.01
../../workdir/run_train_models/raw_electra_hypopt_params_sngp/cola/raw/models/cola/23419
cola  0.8755846585594013  ue.lamb\=0.006 +ue.lamb_intra\=0.01 training.learning_rate\=2e-05 +ue.margin\=2.5 training.num_train_epochs\=4 training.per_device_train_batch_size\=32 +training.weight_decay\=0
../../workdir/run_train_models/raw_electra_hypopt_params_sngp/sst2/raw/models/sst2/23419
sst2  0.9364608076009501  ue.lamb\=0.001 +ue.lamb_intra\=0.002 training.learning_rate\=9e-06 +ue.margin\=0.25 training.num_train_epochs\=7 training.per_device_train_batch_size\=16 +training.weight_decay\=0.1
../../workdir/run_train_models/raw_electra_hypopt_params_sngp/conll/raw/models/conll2003/23419
conll  0.90983606

# Here we extract the same params, but for table in paper

In [23]:
# extract strings for paper table
def to_scientific(value):
    if value == 0 or value == 1.0:
        return str(value)
    else:
        return str(np.format_float_scientific(value, exp_digits=1)).replace('.', '')

def get_param_string(dataset_name, reg_type, sn, params):
    result_string = dataset_name + ' & ' + reg_type + ' & '
    result_string += '+ & ' if sn else '- & '
    #print(params)
    result_string += str(round(params['objective'], 3)) + ' & '
    if reg_type.lower() == 'metric':
        buf = to_scientific(params['lamb_intra']) + '/' + to_scientific(params['lamb']) + ' & ' + str(params['margin']) + ' & '
    elif reg_type.lower() == 'cer':
        buf = to_scientific(params['lamb']) + ' & - & '
    else:
        buf = '- & - & '
    result_string += buf
    pars = ['learning_rate', 'num_train_epochs', 'per_device_train_batch_size', 'weight_decay']
    for par in pars:
        if par in ['num_train_epochs', 'per_device_train_batch_size']:
            result_string += str(params[par]) + ' & '
        else:
            result_string += to_scientific(params[par]) + ' & '
    result_string = result_string[:-3]
    print(result_string)
    return result_string

In [ ]:
path = '../../workdir/run_train_models/distilbert_hypopt/cer_sn'
path = '../../workdir/run_train_models/distilbert_hypopt/metric_sn_fixed_range_last_last_fix/'
path = '../../workdir/run_train_models/distilbert_hypopt/metric_sn_fixed_range_last_last_fix_171/'


regs_names = {'': 'Metric'}
for dataset in datasets:
    for reg in regs:
        buf_path = os.path.join(path, dataset, reg)
        params_string = ''
        for pat, subdirs, files in os.walk(buf_path):
            for name in files:
                if name == 'optimal_hyperparameters.yaml':
                    opt_path = os.path.join(pat, name)
                    with open(opt_path, 'r') as f:
                        data = yaml.safe_load(f)
                    dataset_name = datasets_names[dataset]
                    reg_name = regs_names[reg]
                    sn = True#'sn' in reg
                    res_string = get_param_string(dataset_name, reg_name, sn, data)

{'lamb': 0.006, 'lamb_intra': 0.003, 'learning_rate': 2e-05, 'margin': 5.0, 'num_train_epochs': 10, 'objective': 0.8255179934569248, 'per_device_train_batch_size': 32, 'weight_decay': 0.1}
MRPC & Metric & + & 0.826 & 3e-3/6e-3 & 5.0 & 2e-5 & 10 & 32 & 1e-1
{'lamb': 0.001, 'lamb_intra': 1.0, 'learning_rate': 2e-05, 'margin': 0.5, 'num_train_epochs': 7, 'objective': 0.8026192703461179, 'per_device_train_batch_size': 16, 'weight_decay': 0.01}
CoLA & Metric & + & 0.803 & 1.0/1e-3 & 0.5 & 2e-5 & 7 & 16 & 1e-2
{'lamb': 0.001, 'lamb_intra': 0.05, 'learning_rate': 5e-05, 'margin': 0.25, 'num_train_epochs': 8, 'objective': 0.9020190023752969, 'per_device_train_batch_size': 8, 'weight_decay': 0.1}
SST-2 & Metric & + & 0.902 & 5e-2/1e-3 & 0.25 & 5e-5 & 8 & 8 & 1e-1
{'epoch': 9.0, 'eval_accuracy': 0.9684598315272421, 'eval_f1': 0.8437175493250259, 'eval_loss': 0.19174200296401978, 'eval_precision': 0.824594046008119, 'eval_recall': 0.8637491141034727, 'eval_runtime': 5.3082, 'eval_samples_per_seco

In [28]:
# for deberta
path = '../../workdir/new_final_results/deberta_opt_params/hp_search'
regs_names = {'': 'Metric'}
buf_path = os.path.join(path)#, dataset, reg)
params_string = ''
for pat, subdirs, files in os.walk(buf_path):
    for name in files:
        if name == 'optimal_hyperparameters.yaml':
            opt_path = os.path.join(pat, name)
            print(opt_path.split('/')[-2], opt_path.split('/')[-3])
            dataset_name = opt_path.split('/')[-2]
            reg_name = opt_path.split('/')[-3].split('_')[1]
            with open(opt_path, 'r') as f:
                data = yaml.safe_load(f)
            #dataset_name = datasets_names[dataset]
            #reg_name = regs_names[reg]
            if 'no_sn' in opt_path:
                sn = False
            else:
                sn = True#'sn' in reg
            if reg_name == 'reg':
                reg_name = 'cer'
            res_string = get_param_string(dataset_name, reg_name, sn, data)

cola deberta_metric_no_sn
cola & metric & - & 0.862 & 8e-3/6e-3 & 0.025 & 1e-5 & 12 & 4 & 1e-1
conll2003 deberta_metric_no_sn
conll2003 & metric & - & 0.908 & 2e-2/1e-1 & 0.5 & 3e-5 & 14 & 8 & 1e-2
mrpc deberta_metric_no_sn
mrpc & metric & - & 0.885 & 6e-3/1.0 & 5.0 & 9e-6 & 13 & 8 & 1e-1
sst2 deberta_metric_no_sn
sst2 & metric & - & 0.902 & 6e-3/8e-3 & 0.05 & 7e-6 & 8 & 16 & 1e-1
cola deberta_metric_sn
cola & metric & + & 0.86 & 1.0/3e-3 & 0.1 & 6e-6 & 11 & 4 & 1e-2
conll2003 deberta_metric_sn
conll2003 & metric & + & 0.898 & 2e-3/5e-3 & 5.0 & 7e-5 & 12 & 8 & 1e-2
mrpc deberta_metric_sn
mrpc & metric & + & 0.883 & 8e-3/5e-2 & 2.5 & 2e-5 & 14 & 16 & 1e-1
sst2 deberta_metric_sn
sst2 & metric & + & 0.902 & 6e-3/5e-3 & 0.025 & 5e-5 & 6 & 64 & 1e-1
cola deberta_raw_no_sn
cola & raw & - & 0.86 & - & - & 7e-6 & 13 & 4 & 0
conll2003 deberta_raw_no_sn
conll2003 & raw & - & 0.902 & - & - & 5e-5 & 12 & 32 & 0
mrpc deberta_raw_no_sn
mrpc & raw & - & 0.889 & - & - & 3e-5 & 12 & 4 & 1e-1
sst2 deber

In [24]:
# for electra on old net
path = '../../workdir/new_final_results/electra_20_old_net/opt_params_electra_20_sngp'
regs_names = {'': 'Metric'}
buf_path = os.path.join(path)#, dataset, reg)
params_string = ''
datasets_names = {'mrpc': 'MRPC',
                  'cola': 'CoLA',
                  'sst2': 'SST-2',
                  'conll': 'CoNLL-2003',
                  'conll2003': 'CoNLL-2003'}
regs_names = {'raw': '--', 'raw_sn': '--', 'cer': 'CER', 'cer_sn': 'CER', 'metric': 'Metric', 'metric_sn': 'Metric'}
all_strings = []
for pat, subdirs, files in os.walk(buf_path):
    for name in files:
        if 'optimal_hyperparameters.yaml' in name:
            opt_path = os.path.join(pat, name)
            #print(opt_path.split('/')[-2], opt_path.split('/')[-3])
            if name.split('_')[1] == 'sn':
                dataset_name = name.split('_')[2]
            else:
                dataset_name = name.split('_')[1]
            dataset_name = datasets_names[dataset_name]
            reg_name = name.split('_')[0]
            reg_name = regs_names[reg_name]
            with open(opt_path, 'r') as f:
                data = yaml.safe_load(f)
            #dataset_name = datasets_names[dataset]
            #reg_name = regs_names[reg]
            if 'sn' in opt_path:
                sn = True
            else:
                sn = False#'sn' in reg
            if reg_name == 'reg':
                reg_name = 'cer'
            res_string = get_param_string(dataset_name, reg_name, sn, data)
            all_strings.append(res_string)

MRPC & -- & + & 0.868 & - & - & 1e-4 & 10 & 32 & 1e-2
CoLA & -- & + & 0.876 & - & - & 2e-5 & 4 & 32 & 0
SST-2 & -- & + & 0.936 & - & - & 9e-6 & 7 & 16 & 1e-1
CoNLL-2003 & -- & + & 0.91 & - & - & 5e-5 & 85 & 8 & 1e-2


In [48]:
for string in sorted(all_strings):
    print(string + '\\\\')

CoLA & -- & + & 0.876 & - & - & 3e-5 & 15 & 32 & 1e-1\\
CoLA & -- & - & 0.88 & - & - & 1e-5 & 8 & 4 & 1e-1\\
CoLA & CER & + & 0.88 & 1.0 & - & 3e-5 & 11 & 32 & 1e-1\\
CoLA & CER & - & 0.882 & 1e-2 & - & 9e-6 & 7 & 4 & 1e-2\\
CoLA & Metric & + & 0.868 & 1e-2/1.0 & 0.1 & 3e-5 & 11 & 32 & 1e-1\\
CoLA & Metric & - & 0.878 & 1e-2/2e-2 & 0.25 & 9e-6 & 12 & 4 & 1e-1\\
CoNLL-2003 & -- & + & 0.922 & - & - & 5e-5 & 13 & 8 & 1e-2\\
CoNLL-2003 & -- & - & 0.909 & - & - & 5e-5 & 6 & 8 & 1e-2\\
CoNLL-2003 & CER & + & 0.913 & 1e-1 & - & 5e-5 & 13 & 8 & 1e-2\\
CoNLL-2003 & CER & - & 0.912 & 2e-3 & - & 2e-5 & 15 & 16 & 1e-2\\
CoNLL-2003 & Metric & + & 0.911 & 6e-3/1e-3 & 0.05 & 5e-5 & 15 & 8 & 0\\
CoNLL-2003 & Metric & - & 0.909 & 1e-3/1e-1 & 0.025 & 5e-5 & 13 & 8 & 1e-2\\
MRPC & -- & + & 0.858 & - & - & 3e-5 & 11 & 32 & 1e-1\\
MRPC & -- & - & 0.867 & - & - & 5e-5 & 12 & 32 & 1e-1\\
MRPC & CER & + & 0.871 & 1.0 & - & 3e-5 & 12 & 4 & 0\\
MRPC & CER & - & 0.871 & 2e-1 & - & 5e-5 & 7 & 16 & 1e-2\\
MRPC & M

# Also extract params in similar way for MSD

In [51]:
def get_param_string_msd(dataset_name, reg_type, sn, params):
    result_string = dataset_name + ' & ' + reg_type + ' & '
    result_string += '+ & ' if sn else '- & '
    print(params)
    result_string += str(round(params['objective'], 3)) + ' & '
    if reg_type.lower() == 'metric':
        buf = to_scientific(params['lamb_intra']) + '/' + to_scientific(params['lamb']) + ' & ' + str(params['margin']) + ' & '
    elif reg_type.lower() == 'msd':
        buf = to_scientific(params['lam1']) + '/' + to_scientific(params['lam2']) + ' & ' + str(params['omega']) + ' & '
    elif reg_type.lower() == 'cer':
        buf = to_scientific(params['lamb']) + ' & - & '
    else:
        buf = '- & - & '
    result_string += buf
    pars = ['learning_rate', 'num_train_epochs', 'per_device_train_batch_size', 'weight_decay']
    for par in pars:
        if par in ['num_train_epochs', 'per_device_train_batch_size']:
            result_string += str(params[par]) + ' & '
        else:
            result_string += to_scientific(params[par]) + ' & '
    result_string = result_string[:-3]
    print(result_string)

In [60]:
path = '../../workdir/run_train_models/mixup_electra_hypopt_after_repro/'
path = '../../workdir/run_train_models/mixup_distilbert_hypopt/'
path = '../../workdir/run_train_models/mixup_deberta_hypopt/'

regs = ['']
regs_names = {'': 'MSD'}
for dataset in datasets:
    for reg in regs:
        buf_path = os.path.join(path, dataset, reg)
        params_string = ''
        for pat, subdirs, files in os.walk(buf_path):
            for name in files:
                if name == 'optimal_hyperparameters.yaml':
                    opt_path = os.path.join(pat, name)
                    with open(opt_path, 'r') as f:
                        data = yaml.safe_load(f)
                    dataset_name = datasets_names[dataset]
                    reg_name = regs_names[reg]
                    sn = False#'sn' in reg
                    res_string = get_param_string_msd(dataset_name, reg_name, sn, data)

{'lam1': 0.01, 'lam2': 0.02, 'learning_rate': 9e-06, 'mixup': True, 'num_train_epochs': 12, 'objective': 0.8756815703380589, 'omega': 0.5, 'per_device_train_batch_size': 8, 'self_ensembling': True, 'weight_decay': 0.1}
MRPC & MSD & - & 0.876 & 1e-2/2e-2 & 0.5 & 9e-6 & 12 & 8 & 1e-1
{'lam1': 0.05, 'lam2': 0.003, 'learning_rate': 3e-05, 'mixup': True, 'num_train_epochs': 12, 'objective': 0.8568755846585594, 'omega': 0.65, 'per_device_train_batch_size': 32, 'self_ensembling': True, 'weight_decay': 0}
CoLA & MSD & - & 0.857 & 5e-2/3e-3 & 0.65 & 3e-5 & 12 & 32 & 0
{'lam1': 0.01, 'lam2': 0.003, 'learning_rate': 1e-05, 'mixup': True, 'num_train_epochs': 11, 'objective': 0.9293349168646081, 'omega': 0.95, 'per_device_train_batch_size': 4, 'self_ensembling': True, 'weight_decay': 0.01}
SST-2 & MSD & - & 0.929 & 1e-2/3e-3 & 0.95 & 1e-5 & 11 & 4 & 1e-2
{'lam1': 0.1, 'lam2': 0.005, 'learning_rate': 3e-05, 'mixup': True, 'num_train_epochs': 15, 'objective': 0.9354047424366312, 'omega': 0.95, 'per_d

# Transform params for Deberta

In [5]:
deberta_opt = """MRPC &         - &            - &         0.888620 &           - &  10.000 &       0.000030 &           12 &           4 &          0.10 \\
      MRPC &         - &            + &         0.878935 &           - &   0.010 &       0.000009 &           11 &          16 &          0.10 \\
      MRPC &       CER &            - &         0.880145 &         1.0 &   0.050 &       0.000020 &           10 &          16 &          0.01 \\
      MRPC &       CER &            + &         0.880145 &       0.006 &   1.000 &       0.000020 &           15 &          16 &          0.00 \\
      MRPC &    Metric &            - &         0.877724 &    0.2/0.01 &   0.010 &       0.000009 &           14 &           4 &          0.01 \\
      MRPC &    Metric &            + &         0.871671 & 0.001/0.002 &   0.100 &       0.000030 &            9 &           4 &          0.10 \\
      CoLA &         - &            - &         0.859667 &           - &   5.000 &       0.000007 &           13 &           4 &          0.00 \\
      CoLA &         - &            + &         0.854470 &           - &   0.050 &       0.000007 &            8 &           4 &          0.10 \\
      CoLA &       CER &            - &         0.853950 &         0.2 &   2.500 &       0.000009 &           15 &          16 &          0.01 \\
      CoLA &       CER &            + &         0.857069 &        0.05 &   0.500 &       0.000020 &           11 &          32 &          0.01 \\
      CoLA &    Metric &            - &         0.854470 &   1.0/0.003 &   5.000 &       0.000030 &           11 &          32 &          0.01 \\
      CoLA &    Metric &            + &         0.859148 &  0.02/0.005 &   1.000 &       0.000030 &            6 &          64 &          0.00 \\
     SST-2 &         - &            - &         0.906271 &           - &   0.500 &       0.000030 &            5 &          16 &          0.01 \\
     SST-2 &         - &            + &         0.900990 &           - &   0.100 &       0.000009 &           11 &           8 &          0.10 \\
     SST-2 &       CER &            - &         0.902310 &         0.1 &   0.010 &       0.000006 &            6 &           4 &          0.00 \\
     SST-2 &       CER &            + &         0.901650 &         1.0 &   0.500 &       0.000007 &           12 &          16 &          0.01 \\
     SST-2 &    Metric &            - &         0.899670 &  0.008/0.05 &   0.250 &       0.000010 &           12 &          16 &          0.00 \\
     SST-2 &    Metric &            + &         0.904290 &   0.001/0.1 &   0.050 &       0.000030 &           10 &          16 &          0.00 \\
CoNLL-2003 &         - &            - &         0.901865 &           - &   5.000 &       0.000050 &           12 &          32 &          0.00 \\
CoNLL-2003 &         - &            + &         0.896832 &           - &   0.050 &       0.000050 &            3 &           4 &          0.01 \\
CoNLL-2003 &       CER &            - &         0.898858 &         0.2 &   0.025 &       0.000020 &           13 &           4 &          0.10 \\
CoNLL-2003 &       CER &            + &         0.900651 &        0.05 &   0.100 &       0.000100 &           10 &          16 &          0.00 \\
CoNLL-2003 &    Metric &            - &         0.900081 &  0.002/0.01 &   0.500 &       0.000050 &            7 &           8 &          0.01 \\
CoNLL-2003 &    Metric &            + &         0.901546 &  0.001/0.01 &   1.000 &       0.000070 &           10 &           8 &          0.00 \\"""

In [10]:
strings = deberta_opt.split('\\')

In [36]:
new_strings = []
for string in strings:
    str_data = string.replace("\n", "")
    str_data = str_data.split("&")
    if len(str_data) > 1:
        if '-' in str_data[1]:
            str_data[1] = str_data[1].replace('-', '--')
        for i in range(3, 10):
            str_data[i] = str_data[i].strip()
            try:
                str_data[i] = float(str_data[i])
                if i in [7,8]:
                    str_data[i] = str(int(str_data[i]))
                elif i == 3:
                    str_data[i] = str(round(str_data[i], 3))
                else:
                    str_data[i] = to_scientific(str_data[i])
            except:
                pass
    new_strings.append(' & '.join(str_data))
    print(' & '.join(str_data) + '\\\\')

MRPC  &          --  &             -  & 0.889 & - & 1e+1 & 3e-5 & 12 & 4 & 1e-1\\
      MRPC  &          --  &             +  & 0.879 & - & 1e-2 & 9e-6 & 11 & 16 & 1e-1\\
      MRPC  &        CER  &             -  & 0.88 & 1.0 & 5e-2 & 2e-5 & 10 & 16 & 1e-2\\
      MRPC  &        CER  &             +  & 0.88 & 6e-3 & 1.0 & 2e-5 & 15 & 16 & 0.0\\
      MRPC  &     Metric  &             -  & 0.878 & 0.2/0.01 & 1e-2 & 9e-6 & 14 & 4 & 1e-2\\
      MRPC  &     Metric  &             +  & 0.872 & 0.001/0.002 & 1e-1 & 3e-5 & 9 & 4 & 1e-1\\
      CoLA  &          --  &             -  & 0.86 & - & 5e+0 & 7e-6 & 13 & 4 & 0.0\\
      CoLA  &          --  &             +  & 0.854 & - & 5e-2 & 7e-6 & 8 & 4 & 1e-1\\
      CoLA  &        CER  &             -  & 0.854 & 2e-1 & 25e+0 & 9e-6 & 15 & 16 & 1e-2\\
      CoLA  &        CER  &             +  & 0.857 & 5e-2 & 5e-1 & 2e-5 & 11 & 32 & 1e-2\\
      CoLA  &     Metric  &             -  & 0.854 & 1.0/0.003 & 5e+0 & 3e-5 & 11 & 32 & 1e-2\\
      CoLA